In [12]:
import torch

## load checkpoint

In [18]:
pathe = '/home/rozenberszki/project/wsnsl/output/Own/room1/ckpts/01999.tar'
traj = '/home/rozenberszki/project/wsnsl/Datasets/Replica/room1/traj.txt'

In [19]:
d = torch.load(pathe)

NameError: name 'torch' is not defined

In [20]:
T_e = d['estimate_c2w_list']
#T_e[:,1:3]*= -1

In [21]:
T_e[16]

tensor([[-3.0114e-01,  1.1277e-01,  9.4689e-01, -2.1420e+00],
        [ 9.5358e-01,  3.4882e-02,  2.9912e-01, -4.0176e-01],
        [ 7.0141e-04,  9.9301e-01, -1.1804e-01,  2.1823e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [22]:
import numpy as np
T_wc = np.loadtxt(traj).reshape(-1,4,4)
T_wc[:,1:3]*= -1


In [9]:
np.round(T_e[5].numpy(),3)

array([[-0.174,  0.227,  0.958, -2.211],
       [ 0.985,  0.044,  0.168, -0.463],
       [-0.004,  0.973, -0.231,  0.171],
       [ 0.   ,  0.   ,  0.   ,  1.   ]], dtype=float32)

In [24]:
s = np.ones((4,4), int)
s[[0,0,1,1,2], [1,2,0,3,3]] *=-1

In [33]:
np.round(T_wc[10],2), np.round(T_e[10].numpy(),2),np.round(T_wc[10]- T_e[10].numpy(),3)

(array([[-0.24, -0.16, -0.96, -2.18],
        [-0.97,  0.04,  0.23,  0.43],
        [-0.  ,  0.99, -0.17, -0.21],
        [ 0.  ,  0.  ,  0.  ,  1.  ]]),
 array([[-0.24,  0.17,  0.96, -2.18],
        [ 0.97,  0.04,  0.23, -0.44],
        [-0.  ,  0.98, -0.18,  0.19],
        [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32),
 array([[-2.000e-03, -3.370e-01, -1.914e+00, -0.000e+00],
        [-1.943e+00, -3.000e-03,  2.000e-03,  8.660e-01],
        [ 1.000e-03,  2.000e-03,  9.000e-03, -4.040e-01],
        [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00]]))

In [26]:
T_e = T_e * s

In [32]:
torch.sum(T_e[50][:3,:3]**2)

tensor(3.0000, dtype=torch.float64)

In [34]:
T_e[50][:3,:3]

tensor([[-0.6137,  0.1284, -0.7791],
        [-0.7896, -0.0976,  0.6059],
        [-0.0017,  0.9869,  0.1613]], dtype=torch.float64)

## check decoder update

In [12]:
from src.conv_onet.models.decoder import DenseLayer
from src import config

In [13]:
cfg = config.load_config(  # J:changed it to use our config file including semantics
        '/home/koerner/Project/wsnsl/configs/Own/room1.yaml', "configs/nice_slam_sem.yaml"
    )

In [14]:

model = config.get_model(cfg, nice=True)


In [15]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe

In [10]:
model.semantic_decoder.output_linear = DenseLayer(in_dim=32, out_dim=100, activation='linear')

In [11]:
model

NICE(
  (middle_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): Linear(in_features=32, out_features=32, bias=True)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Linear(in_features=32, out_features=32, bias=True)
      (4): Linear(in_features=32, out_features=32, bias=True)
    )
    (embedder): GaussianFourierFeatureTransform()
    (pts_linears): ModuleList(
      (0): DenseLayer(in_features=93, out_features=32, bias=True)
      (1): DenseLayer(in_features=32, out_features=32, bias=True)
      (2): DenseLayer(in_features=32, out_features=32, bias=True)
      (3): DenseLayer(in_features=125, out_features=32, bias=True)
      (4): DenseLayer(in_features=32, out_features=32, bias=True)
    )
    (output_linear): DenseLayer(in_features=32, out_features=1, bias=True)
  )
  (fine_decoder): MLP(
    (fc_c): ModuleList(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): Linear(in_fe

## TUM RGBD check traj

In [1]:
from src.utils.datasets import get_dataset
import torch
import numpy as np

import argparse
import random

import numpy as np
import torch

from src import config
from src.NICE_SLAM import NICE_SLAM
from src.Segmenter import Segmenter

import os  # J:added
from torch.utils.tensorboard import SummaryWriter  # J: added
import yaml  # J: added
from scripts import gifMaker  # J: added
import argparse

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_traj(path):
    parser = argparse.ArgumentParser(
    description="Arguments for running the NICE-SLAM/iMAP*."
    )
    parser.add_argument("config", type=str, help="Path to config file.")
    parser.add_argument(
        "--input_folder",
        type=str,
        help="input folder, this have higher priority, can overwrite the one in config file",
    )
    parser.add_argument(
        "--output",
        type=str,
        help="output folder, this have higher priority, can overwrite the one in config file",
    )
    nice_parser = parser.add_mutually_exclusive_group(required=False)
    nice_parser.add_argument("--nice", dest="nice", action="store_true")
    nice_parser.add_argument("--imap", dest="nice", action="store_false")
    parser.set_defaults(nice=True)
    args = parser.parse_args(args=[path])
    cfg = config.load_config(  # J:changed it to use our config file including semantics
            args.config, "configs/nice_slam_sem.yaml" if args.nice else "configs/imap.yaml"
        )
    slam = NICE_SLAM(cfg, args)
    frame_reader = get_dataset(cfg, args, cfg["scale"], slam = slam)
    camera_poses = frame_reader.poses
    camera_poses = torch.concat([c[None] for c in camera_poses]).float()
    return camera_poses

In [3]:
traj = get_traj('/home/rozenberszki/project/wsnsl/configs/TUM_RGBD/freiburg2_coke.yaml')

INFO: The output folder is output/TUM_RGBD/rgbd_dataset_freiburg2_coke
INFO: The GT, generated and residual depth/color images can be found under output/TUM_RGBD/rgbd_dataset_freiburg2_coke/mapping_vis/
INFO: The mesh can be found under output/TUM_RGBD/rgbd_dataset_freiburg2_coke/mesh/
INFO: The checkpoint can be found under output/TUM_RGBD/rgbd_dataset_freiburg2_coke/ckpt/


In [5]:
s = np.ones((4, 4), int)
s[[0, 0, 1, 1, 2], [1, 2, 0, 3, 3]] *= -1

In [7]:
traj

tensor([[[ 1.0000e+00, -0.0000e+00, -0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -1.0000e+00, -0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -0.0000e+00, -1.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9999e-01,  2.4111e-03, -2.0922e-03,  9.9391e-04],
         [ 2.4143e-03, -1.0000e+00,  1.5272e-03, -1.3703e-03],
         [-2.0885e-03, -1.5323e-03, -1.0000e+00, -5.8692e-04],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9999e-01,  3.0507e-03, -2.0881e-03,  2.7405e-03],
         [ 3.0640e-03, -9.9997e-01,  6.3907e-03, -2.3808e-03],
         [-2.0685e-03, -6.3970e-03, -9.9998e-01, -7.2207e-04],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        ...,

        [[ 8.7649e-01,  1.0510e-01, -4.6981e-01, -4.1593e-01],
         [ 8.2974e-02, -9.9425e-01, -6.7623e-02, -3.5161e-01],
         [-4.7422e-01,  2.0289e-02, -8.8017e-01,  7.7476e-02],
         [ 0.0000e+00,  0.0000e+00,